In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
df = pd.read_csv('train.csv')

#  Lets Plan

In [3]:
df=df.drop(columns=['Name','Ticket','Fare','Cabin','Parch'])

In [4]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Embarked
0,1,0,3,male,22.0,1,S
1,2,1,1,female,38.0,1,C
2,3,1,3,female,26.0,0,S
3,4,1,1,female,35.0,1,S
4,5,0,3,male,35.0,0,S
...,...,...,...,...,...,...,...
886,887,0,2,male,27.0,0,S
887,888,1,1,female,19.0,0,S
888,889,0,3,female,NaN,1,S
889,890,1,1,male,26.0,0,C


In [5]:
#step-1 train/test/split

X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['Survived']),df['Survived'],test_size=0.2,random_state=42)

In [6]:
X_train

,PassengerId,Pclass,Sex,Age,SibSp,Embarked
331,332,1,male,45.5,0,S
733,734,2,male,23.0,0,S
382,383,3,male,32.0,0,S
704,705,3,male,26.0,1,S
813,814,3,female,6.0,4,S
...,...,...,...,...,...,...
106,107,3,female,21.0,0,S
270,271,1,male,NaN,0,S
860,861,3,male,41.0,2,S
435,436,1,female,14.0,1,S


In [7]:
# imputation transformer
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[3]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[5])
],remainder = 'passthrough')

In [8]:
# One hot encoding
trf2 = ColumnTransformer([
    ('oho_sex_embarked',OneHotEncoder(sparse=False,handle_unknown='ignore'),[2,5])
],remainder = 'passthrough')

In [9]:
#Scaling
# is ma ya seen hai ka above ju colums trnsformer apply kra hai us sa new column add hoy hai isy lya slicing (0,9) hai 
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,9))
])

In [10]:
# Feature Selection
trf4 = SelectKBest(score_func=chi2,k=8)

In [11]:
trf5 = DecisionTreeClassifier()

# Create Pipeline

In [12]:
from sklearn.pipeline import Pipeline

In [13]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
    
])

# Pipeline Vs make_pipeline

Pipeline Vs make_pipeline
Pipeline requires naming of steps, make_pipeline does not.

(Same applies to ColumnTransformer vs make_column_transformer)

In [14]:
# Alternate Syntax
pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

NameError: name 'make_pipeline' is not defined

In [15]:
#train
pipe.fit(X_train,y_train)

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [3]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [5])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('oho_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [2, 5])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 9, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001F22D11D360>)),
                ('trf5', DecisionTreeClassifier())])

In [16]:
# Display Pipeline
from sklearn import set_config

set_config(display='digram')

In [17]:
pipe.named_steps['trf1']

ColumnTransformer(remainder='passthrough',
                  transformers=[('impute_age', SimpleImputer(), [3]),
                                ('impute_embarked',
                                 SimpleImputer(strategy='most_frequent'),
                                 [5])])

In [18]:
# predict
y_predict = pipe.predict(X_test)

In [19]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_predict)

0.5865921787709497

# Cross Validation Using Pipeline

In [20]:
# cross validation using cross_val_score
from sklearn.model_selection import cross_val_score
cross_val_score(pipe,X_train,y_train,cv=5,scoring='accuracy').mean()

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave 

0.6235989362749926

# GridSearch using Pipeline

In [21]:
# gridsearchcv
params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [22]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\python3.10\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [3]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [5])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('oho_sex_embarked',
                                                                         OneHotEnc

In [23]:
grid.best_score_

0.6235989362749926

In [24]:
grid.best_params_

{'trf5__max_depth': 1}

# Exporting the Pipeline

In [25]:
# export 
import pickle
pickle.dump(pipe,open('pipe.pkl','wb'))